<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
#import SparkSession
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('random_forest').getOrCreate()

In [2]:
#read the dataset
df=spark.read.csv('affairs.csv',inferSchema=True,header=True)

In [3]:
#check the shape of the data 
print((df.count(),len(df.columns)))

(6366, 6)


In [4]:
#printSchema
df.printSchema()

root
 |-- rate_marriage: integer (nullable = true)
 |-- age: double (nullable = true)
 |-- yrs_married: double (nullable = true)
 |-- children: double (nullable = true)
 |-- religious: integer (nullable = true)
 |-- affairs: integer (nullable = true)



In [5]:
#view the dataset
df.show(5)

+-------------+----+-----------+--------+---------+-------+
|rate_marriage| age|yrs_married|children|religious|affairs|
+-------------+----+-----------+--------+---------+-------+
|            5|32.0|        6.0|     1.0|        3|      0|
|            4|22.0|        2.5|     0.0|        2|      0|
|            3|32.0|        9.0|     3.0|        3|      1|
|            3|27.0|       13.0|     3.0|        1|      1|
|            4|22.0|        2.5|     0.0|        1|      1|
+-------------+----+-----------+--------+---------+-------+
only showing top 5 rows



In [6]:
#Exploratory Data Analysis
df.describe().select('summary','rate_marriage','age','yrs_married','children','religious').show()

+-------+------------------+------------------+-----------------+------------------+------------------+
|summary|     rate_marriage|               age|      yrs_married|          children|         religious|
+-------+------------------+------------------+-----------------+------------------+------------------+
|  count|              6366|              6366|             6366|              6366|              6366|
|   mean| 4.109644989004084|29.082862079798932| 9.00942507068803|1.3968740182218033|2.4261702796104303|
| stddev|0.9614295945655025| 6.847881883668817|7.280119972766412| 1.433470828560344|0.8783688402641785|
|    min|                 1|              17.5|              0.5|               0.0|                 1|
|    max|                 5|              42.0|             23.0|               5.5|                 4|
+-------+------------------+------------------+-----------------+------------------+------------------+



In [7]:
df.groupBy('affairs').count().show()

+-------+-----+
|affairs|count|
+-------+-----+
|      1| 2053|
|      0| 4313|
+-------+-----+



In [8]:
df.groupBy('rate_marriage').count().show()

+-------------+-----+
|rate_marriage|count|
+-------------+-----+
|            1|   99|
|            3|  993|
|            5| 2684|
|            4| 2242|
|            2|  348|
+-------------+-----+



In [9]:
df.groupBy('rate_marriage','affairs').count().orderBy('rate_marriage','affairs','count',ascending=True).show()

+-------------+-------+-----+
|rate_marriage|affairs|count|
+-------------+-------+-----+
|            1|      0|   25|
|            1|      1|   74|
|            2|      0|  127|
|            2|      1|  221|
|            3|      0|  446|
|            3|      1|  547|
|            4|      0| 1518|
|            4|      1|  724|
|            5|      0| 2197|
|            5|      1|  487|
+-------------+-------+-----+



In [10]:
df.groupBy('religious','affairs').count().orderBy('religious','affairs','count',ascending=True).show()

+---------+-------+-----+
|religious|affairs|count|
+---------+-------+-----+
|        1|      0|  613|
|        1|      1|  408|
|        2|      0| 1448|
|        2|      1|  819|
|        3|      0| 1715|
|        3|      1|  707|
|        4|      0|  537|
|        4|      1|  119|
+---------+-------+-----+



In [11]:
df.groupBy('children','affairs').count().orderBy('children','affairs','count',ascending=True).show()

+--------+-------+-----+
|children|affairs|count|
+--------+-------+-----+
|     0.0|      0| 1912|
|     0.0|      1|  502|
|     1.0|      0|  747|
|     1.0|      1|  412|
|     2.0|      0|  873|
|     2.0|      1|  608|
|     3.0|      0|  460|
|     3.0|      1|  321|
|     4.0|      0|  197|
|     4.0|      1|  131|
|     5.5|      0|  124|
|     5.5|      1|   79|
+--------+-------+-----+



In [12]:
df.groupBy('affairs').mean().show()

+-------+------------------+------------------+------------------+------------------+------------------+------------+
|affairs|avg(rate_marriage)|          avg(age)|  avg(yrs_married)|     avg(children)|    avg(religious)|avg(affairs)|
+-------+------------------+------------------+------------------+------------------+------------------+------------+
|      1|3.6473453482708234|30.537018996590355|11.152459814905017|1.7289332683877252| 2.261568436434486|         1.0|
|      0| 4.329700904242986| 28.39067934152562| 7.989334569904939|1.2388128912589844|2.5045212149316023|         0.0|
+-------+------------------+------------------+------------------+------------------+------------------+------------+



In [13]:
from pyspark.ml.feature import VectorAssembler

In [14]:
df_assembler = VectorAssembler(inputCols=['rate_marriage', 'age', 'yrs_married', 'children', 'religious'], outputCol="features")
df = df_assembler.transform(df)

In [15]:
df.printSchema()

root
 |-- rate_marriage: integer (nullable = true)
 |-- age: double (nullable = true)
 |-- yrs_married: double (nullable = true)
 |-- children: double (nullable = true)
 |-- religious: integer (nullable = true)
 |-- affairs: integer (nullable = true)
 |-- features: vector (nullable = true)



In [16]:
df.select(['features','affairs']).show(10,False)

+-----------------------+-------+
|features               |affairs|
+-----------------------+-------+
|[5.0,32.0,6.0,1.0,3.0] |0      |
|[4.0,22.0,2.5,0.0,2.0] |0      |
|[3.0,32.0,9.0,3.0,3.0] |1      |
|[3.0,27.0,13.0,3.0,1.0]|1      |
|[4.0,22.0,2.5,0.0,1.0] |1      |
|[4.0,37.0,16.5,4.0,3.0]|1      |
|[5.0,27.0,9.0,1.0,1.0] |1      |
|[4.0,27.0,9.0,0.0,2.0] |1      |
|[5.0,37.0,23.0,5.5,2.0]|1      |
|[5.0,37.0,23.0,5.5,2.0]|1      |
+-----------------------+-------+
only showing top 10 rows



In [17]:
#select data for building model
model_df=df.select(['features','affairs'])

In [18]:
train_df,test_df=model_df.randomSplit([0.75,0.25])

In [19]:
train_df.count()

4754

In [20]:
train_df.groupBy('affairs').count().show()

+-------+-----+
|affairs|count|
+-------+-----+
|      1| 1511|
|      0| 3243|
+-------+-----+



In [21]:
test_df.groupBy('affairs').count().show()

+-------+-----+
|affairs|count|
+-------+-----+
|      1|  542|
|      0| 1070|
+-------+-----+



In [22]:
from pyspark.ml.classification import RandomForestClassifier

In [23]:
rf_classifier=RandomForestClassifier(labelCol='affairs',numTrees=50).fit(train_df)

In [24]:
rf_predictions=rf_classifier.transform(test_df)

In [25]:
rf_predictions.show()

+--------------------+-------+--------------------+--------------------+----------+
|            features|affairs|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|[1.0,22.0,2.5,1.0...|      1|[20.3172498668219...|[0.40634499733643...|       1.0|
|[1.0,22.0,2.5,1.0...|      1|[20.3172498668219...|[0.40634499733643...|       1.0|
|[1.0,27.0,2.5,0.0...|      0|[17.4042487985478...|[0.34808497597095...|       1.0|
|[1.0,27.0,2.5,0.0...|      1|[17.8041882006486...|[0.35608376401297...|       1.0|
|[1.0,27.0,2.5,1.0...|      1|[21.4878298682596...|[0.42975659736519...|       1.0|
|[1.0,27.0,6.0,1.0...|      0|[17.6043071785388...|[0.35208614357077...|       1.0|
|[1.0,27.0,6.0,1.0...|      1|[17.6043071785388...|[0.35208614357077...|       1.0|
|[1.0,27.0,6.0,3.0...|      0|[15.7279308574200...|[0.31455861714840...|       1.0|
|[1.0,32.0,13.0,3....|      1|[13.5049413258122...|[0.27009882651624...|    

In [26]:
rf_predictions.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|       0.0| 1332|
|       1.0|  280|
+----------+-----+



In [27]:
rf_predictions.select(['probability','affairs','prediction']).show(10,False)

+----------------------------------------+-------+----------+
|probability                             |affairs|prediction|
+----------------------------------------+-------+----------+
|[0.4063449973364397,0.5936550026635602] |1      |1.0       |
|[0.4063449973364397,0.5936550026635602] |1      |1.0       |
|[0.3480849759709576,0.6519150240290424] |0      |1.0       |
|[0.35608376401297215,0.643916235987028] |1      |1.0       |
|[0.4297565973651939,0.5702434026348061] |1      |1.0       |
|[0.3520861435707777,0.6479138564292222] |0      |1.0       |
|[0.3520861435707777,0.6479138564292222] |1      |1.0       |
|[0.31455861714840133,0.6854413828515987]|0      |1.0       |
|[0.2700988265162441,0.7299011734837558] |1      |1.0       |
|[0.26341884371552154,0.7365811562844785]|1      |1.0       |
+----------------------------------------+-------+----------+
only showing top 10 rows



In [28]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [29]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [30]:
rf_accuracy=MulticlassClassificationEvaluator(labelCol='affairs',metricName='accuracy').evaluate(rf_predictions)

In [31]:
print('The accuracy of RF on test data is {0:.0%}'.format(rf_accuracy))

The accuracy of RF on test data is 72%


In [32]:
print(rf_accuracy)

0.7208436724565757


In [33]:
rf_precision=MulticlassClassificationEvaluator(labelCol='affairs',metricName='weightedPrecision').evaluate(rf_predictions)

In [34]:
print('The precision rate on test data is {0:.0%}'.format(rf_precision))

The precision rate on test data is 71%


In [35]:
rf_precision

0.7097189089124573

In [36]:
rf_auc=BinaryClassificationEvaluator(labelCol='affairs').evaluate(rf_predictions)

In [37]:
print(rf_auc)

0.7387040038624685


In [38]:
# Feature importance

In [39]:
rf_classifier.featureImportances

SparseVector(5, {0: 0.6553, 1: 0.0182, 2: 0.2098, 3: 0.0573, 4: 0.0594})

In [40]:
df.schema["features"].metadata["ml_attr"]["attrs"]

{'numeric': [{'idx': 0, 'name': 'rate_marriage'},
  {'idx': 1, 'name': 'age'},
  {'idx': 2, 'name': 'yrs_married'},
  {'idx': 3, 'name': 'children'},
  {'idx': 4, 'name': 'religious'}]}

In [41]:
# Save the model 

In [44]:
rf_classifier.save("RF_model.pkl")

In [46]:
from pyspark.ml.classification import RandomForestClassificationModel

In [47]:
rf=RandomForestClassificationModel.load("RF_model.pkl")

In [48]:
model_preditions=rf.transform(test_df)

In [49]:
model_preditions.show()

+--------------------+-------+--------------------+--------------------+----------+
|            features|affairs|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|[1.0,22.0,2.5,1.0...|      1|[20.3172498668219...|[0.40634499733643...|       1.0|
|[1.0,22.0,2.5,1.0...|      1|[20.3172498668219...|[0.40634499733643...|       1.0|
|[1.0,27.0,2.5,0.0...|      0|[17.4042487985478...|[0.34808497597095...|       1.0|
|[1.0,27.0,2.5,0.0...|      1|[17.8041882006486...|[0.35608376401297...|       1.0|
|[1.0,27.0,2.5,1.0...|      1|[21.4878298682596...|[0.42975659736519...|       1.0|
|[1.0,27.0,6.0,1.0...|      0|[17.6043071785388...|[0.35208614357077...|       1.0|
|[1.0,27.0,6.0,1.0...|      1|[17.6043071785388...|[0.35208614357077...|       1.0|
|[1.0,27.0,6.0,3.0...|      0|[15.7279308574200...|[0.31455861714840...|       1.0|
|[1.0,32.0,13.0,3....|      1|[13.5049413258122...|[0.27009882651624...|    